# Class Weights
model VGG16 epoch:40

In [ ]:
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Unzip the file from Google Drive into a directory in Colab
!unzip -q /content/drive/MyDrive/FER2013.zip -d /content/

In [ ]:
img_size = 224
batch_size = 16

train_data_dir = '/content/train'  # Update with correct subfolder if necessary
test_data_dir = '/content/test'    # Update with correct subfolder if necessary

datagen_train = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.1,
    channel_shift_range=0.0,
    fill_mode='nearest',
    horizontal_flip=True,
)

train_generator = datagen_train.flow_from_directory(
    train_data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode="sparse",
    shuffle=True,
)

datagen_test = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.1,
    channel_shift_range=0.0,
    fill_mode='nearest',
    horizontal_flip=True,
)

test_generator = datagen_test.flow_from_directory(
    test_data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode="sparse",
    shuffle=False,
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [ ]:
from sklearn.utils import class_weight
import numpy as np

# Compute class weights
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)

# Convert to dictionary
class_weights_dict = dict(enumerate(class_weights))

print(class_weights_dict)

{0: 1.0266046844269623, 1: 9.406618610747051, 2: 1.0010460615781582, 3: 0.5684387684387684, 4: 0.8260394187886635, 5: 0.8491274770777877, 6: 1.293372978330405}


In [ ]:
base_model=ResNet50(weights="imagenet",include_top=False,input_shape=(224,224,3))

# Add layers
x = base_model.output
x = Flatten()(x)
x = Dense(512,activation="relu")(x)
x = Dense(256,activation="relu")(x)
x = Dense(128,activation="relu")(x)
x = Dense(64,activation="relu")(x)
output = Dense(7,activation="softmax")(x)

# final model
model_classWeight=Model(inputs=base_model.input,outputs=output)

# Freeze the base model
for layer in base_model.layers:
    layer.trainable=False

model_classWeight.compile(optimizer=Adam(learning_rate=0.0005),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=10,
    verbose=1,
    min_delta=0.02,
)

checkpoint_callback_val = tf.keras.callbacks.ModelCheckpoint(
    'best_model_val_focal.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

checkpoint_callback_train = tf.keras.callbacks.ModelCheckpoint(
    'best_model_train_focal.keras',
    monitor='accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

In [ ]:
history = model_classWeight.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=40,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    class_weight=class_weights_dict,  # Pass the class weights here
    callbacks=[checkpoint_callback_val, checkpoint_callback_train, reduce_lr],
    verbose=1
)

Epoch 1/40


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1794/1794 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step - accuracy: 0.1120 - loss: 2.1770
Epoch 1: val_accuracy improved from -inf to 0.24805, saving model to best_model_val_ClassWeight.keras

Epoch 1: accuracy improved from -inf to 0.10693, saving model to best_model_train_ClassWeight.keras
1794/1794 ━━━━━━━━━━━━━━━━━━━━ 499s 266ms/step - accuracy: 0.1120 - loss: 2.1769 - val_accuracy: 0.2480 - val_loss: 1.9447 - learning_rate: 5.0000e-04
Epoch 2/40
   1/1794 ━━━━━━━━━━━━━━━━━━━━ 2:18 77ms/step - accuracy: 0.1250 - loss: 1.7992

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 2: val_accuracy did not improve from 0.24805

Epoch 2: accuracy improved from 0.10693 to 0.12500, saving model to best_model_train_ClassWeight.keras
1794/1794 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.1250 - loss: 1.7992 - val_accuracy: 0.0000e+00 - val_loss: 1.9473 - learning_rate: 5.0000e-04
Epoch 3/40
1794/1794 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step - accuracy: 0.1537 - loss: 1.9570
Epoch 3: val_accuracy did not improve from 0.24805

Epoch 3: accuracy improved from 0.12500 to 0.12850, saving model to best_model_train_ClassWeight.keras
1794/1794 ━━━━━━━━━━━━━━━━━━━━ 479s 265ms/step - accuracy: 0.1537 - loss: 1.9570 - val_accuracy: 0.1740 - val_loss: 1.9439 - learning_rate: 5.0000e-04
Epoch 4/40
   1/1794 ━━━━━━━━━━━━━━━━━━━━ 2:20 78ms/step - accuracy: 0.1875 - loss: 1.8272
Epoch 4: val_accuracy did not improve from 0.24805

Epoch 4: accuracy improved from 0.12850 to 0.18750, saving model to best_model_train_ClassWeight.keras
1794/1794 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accu